In [2]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap 
from mpl_toolkits.basemap import Basemap, addcyclic
from my_tools import file_dic, plot_dic
%matplotlib qt

In [3]:
path = '/media/onno/Algemeen/Thesis/'

In [4]:
#load RWP property files
rean_env_gen = xr.open_dataset(path +'era51_mars_env_wledit2000-10000_latavg_v300_79-19_6hourly_anom_from_smoothed04_clim_smoothed.nc').sel(latitude=slice(90,0)).squeeze()
rean_v300_gen = xr.open_dataset(path + 'era51_mars_v300_wledit2000-10000_latavg_79-19_6hourly.nc').sel(latitude=slice(90,0)).squeeze()
rean_env_gen =  rean_env_gen.assign_coords(longitude=(((rean_env_gen.longitude + 180) % 360) - 180)).sortby('longitude')
rean_v300_gen =  rean_v300_gen.assign_coords(longitude=(((rean_v300_gen.longitude + 180) % 360) - 180)).sortby('longitude')

rean_env_gen=rean_env_gen.rename({'longitude':'lon'});rean_env_gen=rean_env_gen.rename({'latitude':'lat'})
rean_v300_gen=rean_v300_gen.rename({'longitude':'lon'});rean_v300_gen=rean_v300_gen.rename({'latitude':'lat'})
# rean_env_gen = rean_env_gen.sel(time=[bool(i) for i in rean_env_gen.time.dt.hour%24==0])
# rean_v300_gen = rean_v300_gen.sel(time=[bool(i) for i in rean_v300_gen.time.dt.hour%24==0])

ERA5RF_env_gen = xr.open_dataset(path+'era5rf_env_wledit2000-10000_latavg_v300_0-240h_12hourly_2x2nh_jan79-dec19.nc',decode_times=False).squeeze()
GFS_env_gen = xr.open_dataset(path+'gefsrf2_env_wledit2000-10000_latavg_v300_control0-252h_6hourly_2x2_dec84-nov19.nc',decode_times=False).squeeze()
ERA5RF_v300_gen = xr.open_dataset(path+'era5rf_v300_wledit2000-10000_latavg_0-240h_12hourly_2x2nh_jan79-dec19.nc',decode_times=False).squeeze()
GFS_v300_gen = xr.open_dataset(path+'gefsrf2_v300_control0-252h_6hourly_2x2_dec84-nov19.nc',decode_times=False).squeeze()

GFS_init_time = pd.Timestamp('1800-01-01')
ERA5RF_init_time = pd.Timestamp('1900-01-01')
GFS_env_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_env_gen.time.values]
GFS_v300_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_v300_gen.time.values]
ERA5RF_env_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_env_gen.time.values]
ERA5RF_v300_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_v300_gen.time.values]

# GFS_env_gen = GFS_env_gen.rename({'lon':'longitude','lat':'latitude'}); GFS_v300_gen = GFS_v300_gen.rename({'lon':'longitude','lat':'latitude'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'lon':'longitude','lat':'latitude'}); ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'lon':'longitude','lat':'latitude'})
GFS_env_gen = GFS_env_gen.assign_coords(lon=(((GFS_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
GFS_v300_gen = GFS_v300_gen.assign_coords(lon=(((GFS_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
ERA5RF_env_gen = ERA5RF_env_gen.assign_coords(lon=(((ERA5RF_env_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
ERA5RF_v300_gen = ERA5RF_v300_gen.assign_coords(lon=(((ERA5RF_v300_gen.lon + 180) % 360) - 180)).sortby('lon').sel(lat=slice(90,0))
# GFS_env_gen = GFS_env_gen.sel(lead=GFS_env_gen.lead.values[[bool(i) for i in GFS_env_gen.lead%24==0]])
# GFS_v300_gen = GFS_v300_gen.sel(lead=GFS_v300_gen.lead.values[[bool(i) for i in GFS_v300_gen.lead%24==0]])
# ERA5RF_env_gen = ERA5RF_env_gen.sel(lead=ERA5RF_env_gen.lead.values[[bool(i) for i in ERA5RF_env_gen.lead%24==0]])
# # ERA5RF_v300_gen = ERA5RF_v300_gen.sel(lead=ERA5RF_v300_gen.lead.values[[bool(i) for i in ERA5RF_v300_gen.lead%24==0]])

# GFS_env_gen['lead'] = GFS_env_gen.lead.values//24
# GFS_v300_gen['lead'] = GFS_v300_gen.lead.values//24
# ERA5RF_env_gen['lead'] = ERA5RF_env_gen.lead.values//24
# # ERA5RF_v300_gen['lead'] = ERA5RF_v300_gen.lead.values//24

# GFS_env_gen = GFS_env_gen.rename({'latitude':'lat','longitude':'lon'})
# GFS_v300_gen = GFS_v300_gen.rename({'latitude':'lat','longitude':'lon'})
# ERA5RF_env_gen = ERA5RF_env_gen.rename({'latitude':'lat','longitude':'lon'})
# # ERA5RF_v300_gen = ERA5RF_v300_gen.rename({'latitude':'lat','longitude':'lon'})





In [13]:
coordinatez = [
(54,46,6,14)] #Germany
file = '/GFS_T850/rank_forecast/seasonal/{}_forecasts{}_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead-day_{}.txt'
dataset_type = '_split_dataset'
eventz = ['persistent_hw','persistent_cw','short_hw','short_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave','Short Heatwave','Short Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
                     'persistent_cw': np.arange(24,61,2),
                     'short_hw':np.arange(24,61,2),
                     'short_cw':np.arange(24,61,2)},
               'JJA':{'persistent_hw':np.arange(16,61,2),
                     'persistent_cw': np.arange(16,61,2),
                     'short_hw':np.arange(16,61,2),
                     'short_cw':np.arange(16,61,2)}}

lead_day = 5
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    for i,event in enumerate(eventz):
        datez_event = np.load(path + 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
        for model in modelz:
            for season in seasonz:
                df_good = pd.read_csv(path+file.format('good',dataset_type,event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_bad = pd.read_csv(path+file.format('bad',dataset_type,event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_good.index = pd.to_datetime(df_good.index)
                df_bad.index = pd.to_datetime(df_bad.index)
                reanz_env_good = []
                reanz_v300_good = []
                for date in df_good.index:
                    time_begin = date - pd.Timedelta(10,'days')
                    time_end = date + pd.Timedelta(10,'days')
                    rean_env_good_sub = rean_env_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    rean_v300_good_sub = rean_v300_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    rean_env_good_sub = rean_env_good_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_good_sub.time)))
                    rean_v300_good_sub = rean_v300_good_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_good_sub.time)))

                    reanz_env_good.append(rean_env_good_sub)
                    reanz_v300_good.append(rean_v300_good_sub)
                rean_env_good = xr.concat(reanz_env_good,dim='time').groupby('time').mean().mean('lat')
                rean_v300_good = xr.concat(reanz_v300_good,dim='time').groupby('time').mean().mean('lat')
                reanz_env_bad = []
                reanz_v300_bad = []
                for date in df_bad.index:
                    time_begin = date - pd.Timedelta(10,'days')
                    time_end = date + pd.Timedelta(10,'days')
                    rean_env_bad_sub = rean_env_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    rean_v300_bad_sub = rean_v300_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    rean_env_bad_sub = rean_env_bad_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_bad_sub.time)))
                    rean_v300_bad_sub = rean_v300_bad_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_bad_sub.time)))

                    reanz_env_bad.append(rean_env_bad_sub)
                    reanz_v300_bad.append(rean_v300_bad_sub)
                rean_env_bad = xr.concat(reanz_env_bad,dim='time').groupby('time').mean().mean('lat')
                rean_v300_bad = xr.concat(reanz_v300_bad,dim='time').groupby('time').mean().mean('lat')
                fig,axz=plt.subplots(2,1,figsize=(16,9))
                ax1 =axz.flat[0];ax2=axz.flat[1]
                lonz = rean_env_good.lon.values
                dayz = rean_env_good.time.values

                clr_levels = np.concatenate((np.arange(-30,-5,2),np.arange(6,31,2)))
                contour_levels = contour_dic[season][event]
                im = ax1.contourf(lonz,dayz,rean_v300_good.v.values,cmap='bwr',levels=clr_levels,extend='both')
                ax1.contour(lonz,dayz,rean_env_good.v.values,levels=contour_levels)
                ax1.set_title('Good Forecasts N = {}'.format(len(df_good.index)))
                ax1.axvline(x=6,color='grey');ax1.axvline(x=14,color='grey')
                ax1.axhline(y=0,color='grey')
                ax1.set_ylabel('Days before/after onset event')


                ax2.contourf(lonz,dayz,rean_v300_bad.v.values,cmap='bwr',levels=clr_levels,extend='both')
                ax2.contour(lonz,dayz,rean_env_bad.v.values,levels=contour_levels)

                ax2.set_title('Bad Forecasts N = {}'.format(len(df_bad.index)))
                ax2.axvline(x=6,color='grey');ax2.axvline(x=14,color='grey')
                ax2.axhline(y=0,color='grey')
                ax2.set_ylabel('Days before/after onset event')
                ax2.set_xlabel('Longitude [°]')
                fig.suptitle('{} {} {}'.format(event_titlez[i],season,model))

                fig.subplots_adjust(right=0.8)
                cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
                cbar = fig.colorbar(im, cax=cbar_ax)
                cbar.ax.get_yaxis().set_ticks([-30,-20,-10,10,20,30])
                cbar.ax.set_yticklabels([-30,-20,-10,10,20,30])
                cbar.ax.set_ylabel('v (m/s)')

                fig.savefig(path+'fcst_composite/hovmoller/hovmoller_reanalysis{}_{}_{}_lon_{}_{}_lat_{}_{}_{}.png'.format(dataset_type,
                                                                                                                model,event,
                                                                                                                lon_0,lon_1,
                                                                                                                lat_0,lat_1,
                                                                                                                season))
                plt.close(fig)







Hovmoller Forecast vs Reanalysisn for meridional wind

In [64]:
coordinatez = [
(54,46,6,14)] #Germany
file = '/GFS_T850/rank_forecast/{}_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'
eventz = ['persistent_hw','persistent_cw','short_hw','short_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave','Short Heatwave','Short Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
modelz_data = [GFS_v300_gen,ERA5RF_v300_gen]
contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
                     'persistent_cw': np.arange(24,61,2),
                     'short_hw':np.arange(24,61,2),
                     'short_cw':np.arange(24,61,2)},
               'JJA':{'persistent_hw':np.arange(16,61,2),
                     'persistent_cw': np.arange(16,61,2),
                     'short_hw':np.arange(16,61,2),
                     'short_cw':np.arange(16,61,2)}}

lead_day = 3
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    for i,event in enumerate(eventz):
        datez_event = np.load(path + 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
        for j,model in enumerate(modelz):
            fcst_data = modelz_data[j]
            for season in seasonz:
                df_good = pd.read_csv(path+file.format('good',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_bad = pd.read_csv(path+file.format('bad',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_good.index = pd.to_datetime(df_good.index)
                df_bad.index = pd.to_datetime(df_bad.index)
                fcstz_v300_good = []
                reanz_v300_good = []
                for date in df_good.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
#                     rean_env_good_sub = rean_env_gen.sel(lat=slice(70,40),
#                                                    time=slice(time_begin,time_end))
                    rean_v300_good_sub = rean_v300_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    if model=='ERA5RF':
                        rean_v300_good_sub = rean_v300_good_sub.sel(time = rean_v300_good_sub.time.values[[bool(i) for i in rean_v300_good_sub.time.dt.hour%12==0]])
                    rean_v300_good_sub = rean_v300_good_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_v300_good_sub.time)))
#                     rean_v300_good_sub = rean_v300_good_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_good_sub.time)))
                    fcst_v300_good_sub = fcst_data.sel(lat=slice(70,40),time=time_begin,lead=slice(0,240))
                    fcst_v300_good_sub = fcst_v300_good_sub.assign_coords(lead=np.linspace(-lead_day,-lead_day+10,len(rean_v300_good_sub.time)))

#                     reanz_env_good.append(rean_env_good_sub)
                    reanz_v300_good.append(rean_v300_good_sub)
                    fcstz_v300_good.append(fcst_v300_good_sub)
#                 rean_env_good = xr.concat(reanz_env_good,dim='time').groupby('time').mean().mean('lat')
                rean_v300_good = xr.concat(reanz_v300_good,dim='time').groupby('time').mean().mean('lat')
                fcst_v300_good = xr.concat(fcstz_v300_good,dim='lead').groupby('lead').mean().mean('lat')
                fcstz_v300_bad = []
                reanz_v300_bad = []
                for date in df_bad.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
#                     rean_env_bad_sub = rean_env_gen.sel(lat=slice(70,40),
#                                                    time=slice(time_begin,time_end))
                    rean_v300_bad_sub = rean_v300_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    if model=='ERA5RF':
                        rean_v300_bad_sub = rean_v300_bad_sub.sel(time = rean_v300_bad_sub.time.values[[bool(i) for i in rean_v300_bad_sub.time.dt.hour%12==0]])
                    rean_v300_bad_sub = rean_v300_bad_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_v300_bad_sub.time)))
#                     rean_v300_bad_sub = rean_v300_bad_sub.assign_coords(time=np.linspace(-10,10,len(rean_env_bad_sub.time)))
                    fcst_v300_bad_sub = fcst_data.sel(lat=slice(70,40),time=time_begin,lead=slice(0,240))
                    fcst_v300_bad_sub = fcst_v300_bad_sub.assign_coords(lead=np.linspace(-lead_day,-lead_day+10,len(rean_v300_bad_sub.time)))

#                     reanz_env_bad.append(rean_env_bad_sub)
                    reanz_v300_bad.append(rean_v300_bad_sub)
                    fcstz_v300_bad.append(fcst_v300_bad_sub)
#                 rean_env_bad = xr.concat(reanz_env_bad,dim='time').groupby('time').mean().mean('lat')
                rean_v300_bad = xr.concat(reanz_v300_bad,dim='time').groupby('time').mean().mean('lat')
                fcst_v300_bad = xr.concat(fcstz_v300_bad,dim='lead').groupby('lead').mean().mean('lat')
                fig,axz=plt.subplots(2,1,figsize=(16,9))
                ax1 =axz.flat[0];ax2=axz.flat[1]
                lonz = rean_v300_bad.lon.values
                dayz = rean_v300_bad.time.values

                clr_levels = np.concatenate((np.arange(-30,-5,2),np.arange(6,31,2)))
                contour_levels = contour_dic[season][event]
                im = ax1.contourf(lonz,dayz,rean_v300_good.v.values,cmap='bwr',levels=clr_levels,extend='both')
                ax1.contour(lonz,dayz,fcst_v300_good.v.values,levels=clr_levels)
                ax1.set_title('Good Forecasts N = {}'.format(len(df_good.index)))
                ax1.axvline(x=6,color='grey');ax1.axvline(x=14,color='grey')
                ax1.axhline(y=0,color='grey')
                ax1.set_ylabel('Days before/after onset event')


                ax2.contourf(lonz,dayz,rean_v300_bad.v.values,cmap='bwr',levels=clr_levels,extend='both')
                ax2.contour(lonz,dayz,fcst_v300_bad.v.values,levels=clr_levels)

                ax2.set_title('Bad Forecasts N = {}'.format(len(df_bad.index)))
                ax2.axvline(x=6,color='grey');ax2.axvline(x=14,color='grey')
                ax2.axhline(y=0,color='grey')
                ax2.set_ylabel('Days before/after onset event')
                ax2.set_xlabel('Longitude [°]')
                fig.suptitle('{} {} {}'.format(event_titlez[i],season,model))

                fig.subplots_adjust(right=0.8)
                cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
                cbar = fig.colorbar(im, cax=cbar_ax)
                cbar.ax.get_yaxis().set_ticks([-30,-20,-10,10,20,30])
                cbar.ax.set_yticklabels([-30,-20,-10,10,20,30])
                cbar.ax.set_ylabel('v (m/s)')

                fig.savefig(path+'fcst_composite/hovmoller/forecast/hovmoller_forecast_{}_{}_lon_{}_{}_lat_{}_{}_{}_lead_day_{}.png'.format(
                                                                                                                model,event,
                                                                                                                lon_0,lon_1,
                                                                                                                lat_0,lat_1,
                                                                                                                season,lead_day))
                plt.close(fig)







Hovmoller Forecast vs Reanalysisn for RWP envelope

In [5]:
coordinatez = [
(54,46,6,14)] #Germany
file = '/GFS_T850/rank_forecast/{}_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'
eventz = ['persistent_hw','persistent_cw','short_hw','short_cw']
event_titlez = ['Persistent Heatwave','Persistent Coldwave','Short Heatwave','Short Coldwave']
# fcstz_env = [GFS_env_gen]
# fcstz_v300 = [GFS_v300_gen]
# modelz = ['GFS','ERA5RF']
seasonz = ['JJA','DJF']
modelz = ['GFS','ERA5RF']
modelz_data = [GFS_env_gen,ERA5RF_env_gen]
contour_dic = {'DJF':{'persistent_hw':np.arange(24,61,2),
                     'persistent_cw': np.arange(24,61,2),
                     'short_hw':np.arange(24,61,2),
                     'short_cw':np.arange(24,61,2)},
               'JJA':{'persistent_hw':np.arange(16,61,2),
                     'persistent_cw': np.arange(16,61,2),
                     'short_hw':np.arange(16,61,2),
                     'short_cw':np.arange(16,61,2)}}

lead_day = 5
for lat_0,lat_1,lon_0,lon_1 in coordinatez:
    for i,event in enumerate(eventz):
        datez_event = np.load(path + 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
        for j,model in enumerate(modelz):
            fcst_data = modelz_data[j]
            for season in seasonz:
                df_good = pd.read_csv(path+file.format('good',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_bad = pd.read_csv(path+file.format('bad',event,lon_0,lon_1,lat_0,lat_1,season,model,lead_day),index_col=0)
                df_good.index = pd.to_datetime(df_good.index)
                df_bad.index = pd.to_datetime(df_bad.index)
                fcstz_env_good = []
                reanz_env_good = []
                for date in df_good.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
                    rean_env_good_sub = rean_env_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    if model=='ERA5RF':
                        rean_env_good_sub = rean_env_good_sub.sel(time = rean_env_good_sub.time.values[[bool(i) for i in rean_env_good_sub.time.dt.hour%12==0]])
                    rean_env_good_sub = rean_env_good_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_env_good_sub.time)))
                    fcst_env_good_sub = fcst_data.sel(lat=slice(70,40),time=time_begin,lead=slice(0,240))
                    fcst_env_good_sub = fcst_env_good_sub.assign_coords(lead=np.linspace(-lead_day,-lead_day+10,len(rean_env_good_sub.time)))

                    reanz_env_good.append(rean_env_good_sub)
                    fcstz_env_good.append(fcst_env_good_sub)
                rean_env_good = xr.concat(reanz_env_good,dim='time').groupby('time').mean().mean('lat')
                fcst_env_good = xr.concat(fcstz_env_good,dim='lead').groupby('lead').mean().mean('lat')
                fcstz_env_bad = []
                reanz_env_bad = []
                for date in df_bad.index:
                    time_begin = date - pd.Timedelta(lead_day,'days')
                    time_end = time_begin + pd.Timedelta(10,'days')
                    rean_env_bad_sub = rean_env_gen.sel(lat=slice(70,40),
                                                   time=slice(time_begin,time_end))
                    if model=='ERA5RF':
                        rean_env_bad_sub = rean_env_bad_sub.sel(time = rean_env_bad_sub.time.values[[bool(i) for i in rean_env_bad_sub.time.dt.hour%12==0]])
                    rean_env_bad_sub = rean_env_bad_sub.assign_coords(time=np.linspace(-lead_day,-lead_day+10,len(rean_env_bad_sub.time)))
                    fcst_env_bad_sub = fcst_data.sel(lat=slice(70,40),time=time_begin,lead=slice(0,240))
                    fcst_env_bad_sub = fcst_env_bad_sub.assign_coords(lead=np.linspace(-lead_day,-lead_day+10,len(rean_env_bad_sub.time)))

                    reanz_env_bad.append(rean_env_bad_sub)
                    fcstz_env_bad.append(fcst_env_bad_sub)
                rean_env_bad = xr.concat(reanz_env_bad,dim='time').groupby('time').mean().mean('lat')
                fcst_env_bad = xr.concat(fcstz_env_bad,dim='lead').groupby('lead').mean().mean('lat')
                fig,axz=plt.subplots(2,1,figsize=(16,9))
                ax1 =axz.flat[0];ax2=axz.flat[1]
                lonz = rean_env_bad.lon.values
                dayz = rean_env_bad.time.values

                clr_levels = np.concatenate((np.arange(-30,-5,2),np.arange(6,31,2)))
                contour_levels = contour_dic[season][event]
                im = ax1.contourf(lonz,dayz,rean_env_good.v.values,cmap='ocean',levels=contour_levels)
                ax1.contour(lonz,dayz,fcst_env_good.v.values,levels=contour_levels)
                ax1.set_title('Good Forecasts N = {}'.format(len(df_good.index)))
                ax1.axvline(x=6,color='grey');ax1.axvline(x=14,color='grey')
                ax1.axhline(y=0,color='grey')
                ax1.set_ylabel('Days before/after onset event')


                ax2.contourf(lonz,dayz,rean_env_bad.v.values,cmap='ocean',levels=contour_levels)
                ax2.contour(lonz,dayz,fcst_env_bad.v.values,levels=contour_levels)

                ax2.set_title('Bad Forecasts N = {}'.format(len(df_bad.index)))
                ax2.axvline(x=6,color='grey');ax2.axvline(x=14,color='grey')
                ax2.axhline(y=0,color='grey')
                ax2.set_ylabel('Days before/after onset event')
                ax2.set_xlabel('Longitude [°]')
                fig.suptitle('{} {} {}'.format(event_titlez[i],season,model))

                fig.subplots_adjust(right=0.8)
                cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
                cbar = fig.colorbar(im, cax=cbar_ax)
#                 cbar.ax.get_yaxis().set_ticks([-30,-20,-10,10,20,30])
#                 cbar.ax.set_yticklabels([-30,-20,-10,10,20,30])
                cbar.ax.set_ylabel('v (m/s)')

                fig.savefig(path+'fcst_composite/hovmoller/forecast/hovmoller_envelope_forecast_{}_{}_lon_{}_{}_lat_{}_{}_{}_lead_day_{}.png'.format(
                                                                                                                model,event,
                                                                                                                lon_0,lon_1,
                                                                                                                lat_0,lat_1,
                                                                                                                season,lead_day))
                plt.close(fig)





